In [ ]:
import os
import json
import random
import pandas as pd
from collections import defaultdict

# ===================== 配置区 =====================
# INPUT_JSONL = '/home/vcj9002/jianshu/chengxuan/ProgressLM/data/train/rl/text_rl_all_3to7_clean.jsonl'
INPUT_JSONL = '/home/vcj9002/jianshu/chengxuan/ProgressLM/data/train/text_demo/new/rl_text_clean_other.jsonl'
OUTPUT_DIR  = '/home/vcj9002/jianshu/chengxuan/ProgressLM/data/train/text_demo/new'
SFT_JSONL   = os.path.join(OUTPUT_DIR, 'new_text_negative_test.jsonl')
RL_JSONL    = os.path.join(OUTPUT_DIR, 'rl_text_clean_other_2.jsonl')
SPLIT_RATIO = 0.15   # SFT 占 10%，RL 占 90%
SEED = 42
# =================================================

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def extract_action_type(id_string: str):
    parts = id_string.split('/')
    if len(parts) >= 2:
        return parts[1]
    return None

def load_jsonl(jsonl_path: str):
    data = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    print(f"✓ 成功加载 {len(data)} 条数据")
    return data

def group_by_trajectory(data):
    """
    将数据按完整 id（即轨迹）聚合。
    返回: {action_type: {trajectory_id: [items...] } }
    """
    grouped = defaultdict(lambda: defaultdict(list))
    for item in data:
        tid = item.get('id', '')
        act = extract_action_type(tid)
        if act:
            grouped[act][tid].append(item)
    return grouped

def dump_jsonl(data, out_path: str):
    with open(out_path, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

def main():
    random.seed(SEED)
    ensure_dir(OUTPUT_DIR)

    # 1) 加载数据
    data = load_jsonl(INPUT_JSONL)

    # 2) 按类别 & 轨迹分组
    grouped = group_by_trajectory(data)
    print(f"✓ 共发现 {len(grouped)} 个类别（基于 extract_action_type）\n")

    # 3) 分层划分 SFT / RL
    sft_data, rl_data = [], []

    category_stats = []  # 用于打印统计
    for act, traj_map in grouped.items():
        traj_ids = list(traj_map.keys())
        random.shuffle(traj_ids)

        # 计算 SFT 轨迹数量，确保至少有1个
        sft_count = max(1, int(len(traj_ids) * SPLIT_RATIO))
        
        # 特殊情况处理：如果只有1个轨迹
        if len(traj_ids) == 1:
            # 选项1：同时放入SFT和RL（数据会重复）
            sft_traj = traj_ids[:]
            rl_traj = traj_ids[:]
            
            # 选项2：只放入SFT（如果不想要数据重复，使用这个）
            # sft_traj = traj_ids[:]
            # rl_traj = []
        else:
            # 正常情况：按比例分配，但确保SFT至少有1个
            sft_traj = traj_ids[:sft_count]
            rl_traj = traj_ids[sft_count:]

        # 收集数据
        sft_items = [item for tid in sft_traj for item in traj_map[tid]]
        rl_items = [item for tid in rl_traj for item in traj_map[tid]]

        sft_data.extend(sft_items)
        rl_data.extend(rl_items)

        # 记录统计
        category_stats.append({
            '类别': act,
            '总轨迹数': len(traj_ids),
            'SFT_轨迹数': len(sft_traj),
            'RL_轨迹数': len(rl_traj),
            'SFT_样本数': len(sft_items),
            'RL_样本数': len(rl_items),
            '重复': '是' if len(traj_ids) == 1 and len(rl_traj) > 0 else '否'
        })

    # 4) 写出文件
    dump_jsonl(sft_data, SFT_JSONL)
    dump_jsonl(rl_data, RL_JSONL)

    # 5) 打印统计信息
    print("\n====== 数据集划分完成 ======")
    print(f"总类别数: {len(category_stats)}")
    print(f"目标比例 SFT : RL = {SPLIT_RATIO:.0%} : {1-SPLIT_RATIO:.0%}\n")

    print("类别分布统计:")
    print("-" * 80)
    print(f"{'类别':20s} | {'总轨迹':6s} | {'SFT轨迹':7s} {'RL轨迹':7s} | {'SFT样本':7s} {'RL样本':7s} | {'重复':4s}")
    print("-" * 80)
    
    total_trajs = 0
    single_traj_categories = 0
    
    for stat in category_stats:
        print(f"{stat['类别']:20s} | {stat['总轨迹数']:6d} | "
              f"{stat['SFT_轨迹数']:7d} {stat['RL_轨迹数']:7d} | "
              f"{stat['SFT_样本数']:7d} {stat['RL_样本数']:7d} | "
              f"{stat['重复']:4s}")
        total_trajs += stat['总轨迹数']
        if stat['总轨迹数'] == 1:
            single_traj_categories += 1
    
    print("-" * 80)
    print(f"汇总: SFT 总样本数: {len(sft_data)}   RL 总样本数: {len(rl_data)}")
    print(f"      总轨迹数: {total_trajs}   只有1个轨迹的类别数: {single_traj_categories}")
    
    # 验证每个类别都有SFT数据
    missing_sft = [stat['类别'] for stat in category_stats if stat['SFT_轨迹数'] == 0]
    if missing_sft:
        print(f"\n⚠️ 警告：以下类别没有SFT数据: {missing_sft}")
    else:
        print(f"\n✅ 验证通过：所有 {len(category_stats)} 个类别都有至少1个轨迹在SFT中")
    
    print(f"\n👉 输出文件已生成:\n   - {SFT_JSONL}\n   - {RL_JSONL}\n")

if __name__ == '__main__':
    main()

✓ 成功加载 15240 条数据
✓ 共发现 121 个类别（基于 extract_action_type）


====== 数据集划分完成 ======
总类别数: 121
目标比例 SFT : RL = 20% : 80%

类别分布统计:
--------------------------------------------------------------------------------
类别                   | 总轨迹    | SFT轨迹   RL轨迹    | SFT样本   RL样本    | 重复  
--------------------------------------------------------------------------------
10_packplate_2       |     17 |       3      14 |      27     127 | 否   
11_brushcup_2        |     31 |       6      25 |      38     173 | 否   
14_steamcorn         |     35 |       7      28 |      44     164 | 否   
12_packcup           |      7 |       1       6 |       9      56 | 否   
15_steamegg_2        |      4 |       1       3 |       9      27 | 否   
18_makebread_2       |      1 |       1       1 |       3       3 | 是   
19_takebread         |     31 |       6      25 |      41     153 | 否   
16_steampotato_2     |     32 |       6      26 |      28     146 | 否   
18_makebread         |      1 |       1       1 |       3